In [2]:
import miecode
import numpy as np

from mie import compute_table, get_phase_function
from refraction import ice_index_of_refraction
from checks import check_errcode

In [3]:
import xarray as xr

In [4]:
def freq_to_wavelength(mu):
    """Convert frequency in GHz to wavelenght in Microns.
    """
    return 1e6*(299792458.0/(mu*1e9))

In [5]:
mu = 94.0
T = 250.0

real, imag = ice_index_of_refraction(mu, T)
refractive_index = real - 1j*imag

table = compute_table(
    freq_to_wavelength(mu), 
    refractive_index,
    1e2,
    5e4,
)

In [6]:
table

<xarray.Dataset>
Dimensions:       (legendre_index: 239, radius: 710, stokes_index: 6)
Coordinates:
  * radius        (radius) float32 2.0 7.075899 ... 49911.973 50011.973
  * stokes_index  (stokes_index) <U3 'P11' 'P22' 'P33' 'P44' 'P12' 'P34'
Dimensions without coordinates: legendre_index
Data variables:
    extinction    (radius) float32 0.00012509458 0.00554336 ... 16699574000.0
    scatter       (radius) float32 1.4210233e-09 2.7869685e-06 ... 15382108000.0
    nleg          (radius) int32 4 4 6 6 6 6 6 6 ... 234 234 236 236 236 236 238
    legendre      (stokes_index, legendre_index, radius) float32 1.4210232e-09 ... 4.796635
Attributes:
    particle_type:               Ice
    refractive_index:            (array([1.77973945]), array([-0.00157939]))
    refractive_index_source:     refraction.py
    units:                       ['Radius [micron]', 'Wavelength [micron]']
    wavelength_band:             (3189.281468085106, 3189.281468085106)
    wavelength_center:           3189.281494140625
    wavelength_averaging:        None
    wavelength_resolution:       -1
    maximum_legendre:            238
    minimum_effective_radius:    100.0
    maximum_integration_radius:  50000.0

In [7]:
%matplotlib qt
import pylab as py
py.plot(table.radius, table.extinction/(np.pi*table.radius**2))

In [8]:
from size_distribution import gamma
size_dist = gamma(table.radius, reff=1000.0, veff=0.1, normalization='number_concentration')

In [9]:
py.figure()
py.plot(table.radius, size_dist[:,0])

In [10]:
import size_distribution
size_distributions = size_distribution.get_size_distribution_grid(table.radius, size_distribution_function=gamma,
                               particle_density=1.0, radius_units='micron',
                               reff=np.linspace(1e3, 2e3,5),
                          veff=np.linspace(0.1,1.0,2))

In [13]:
py.figure()
size_distributions.number_density[:,0,-1].plot(yscale='log')

In [16]:

poly_table = mie.get_poly_table(size_distributions, table)

In [23]:
phases = get_phase_function(poly_table.legcoef, angles=np.linspace(0.0,180.0,300))#angles=np.array([180.0]))

In [25]:
py.figure()
phases[0,:,0,0].plot(yscale='log')